In [ ]:
##########################################################################
# STAGE 1 - DATA PREPARATION
# - Data Cleaning of EMPA & Laser Data
# - Saving the initial 'clean' data for Data Visualization
# - The Data Visualization script is on its own
#######################################################################

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
import numpy as np
import re # handling regex
import matplotlib.pyplot as plt
import seaborn as sns
# import random
from pathlib import Path
import os

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Setting Paths for the files generated
ROOT = Path("/content/drive/MyDrive/Intro2Prog")
DATA_RAW = ROOT / "Data_Raw"
DATA_CLEAN = ROOT / "Data_Clean"

DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_CLEAN.mkdir(parents=True, exist_ok=True)

# Load raw data
#!ls -al '/content/drive/MyDrive/'

Mounted at /content/drive


In [ ]:
# Load Data file
df = pd.read_excel(DATA_RAW / "Ali New Spreadsheet.xlsx", sheet_name = 'Ali Spreadsheet')

# Inspecting the columen names
print('Source File:', df.shape)
print('\nColumns in Source File:')
print(df.columns.tolist())

Source File: (653, 58)

Columns in Source File:
['SAMPLE', 'Lab', 'CaO', 'SiO2', 'Cr2O3', 'Na2O', 'TiO2', 'MnO', 'MgO', 'FeO', 'Al2O3', 'K2O', 'Mg#', 'Li', 'Be', 'B', 'Mg', 'Si', 'Ca', 'Ca.1', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Co', 'Ni', 'Cu', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'Pb', 'Th', 'U', 'C', 'M', 'R', 'T']


In [ ]:
# Noticed duplicated Ca columns in the dataset
# perform temporary check - on duplicated columns
if 'Ca.1' in df.columns:
  print('\nDuplicated Ca columns found')

# Checking on the percentage of missing entries in both columns
# to decide on which to keep
  ca_missing = df['Ca'].isna().mean()*100
  ca1_missing = df['Ca.1'].isna().mean()*100

  print(f'Ca missing: {ca_missing:.2f}%')
  print(f'Ca.1 missing: {ca1_missing:.2f}%')


Duplicated Ca columns found
Ca missing: 71.06%
Ca.1 missing: 71.06%


In [ ]:
# Both Ca and Ca.1 - has the same no. of missing entries
# so keeping the first one - Ca and dropping the second
if 'Ca.1' in df.columns:
  df = df.drop(columns=['Ca.1'])
  print('Ca.1 - Duplicated Ca column is dropped')
  print('Source File:', df.shape)
  print(df.columns.tolist())

Ca.1 - Duplicated Ca column is dropped
Source File: (653, 57)
['SAMPLE', 'Lab', 'CaO', 'SiO2', 'Cr2O3', 'Na2O', 'TiO2', 'MnO', 'MgO', 'FeO', 'Al2O3', 'K2O', 'Mg#', 'Li', 'Be', 'B', 'Mg', 'Si', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Co', 'Ni', 'Cu', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'Pb', 'Th', 'U', 'C', 'M', 'R', 'T']


In [ ]:
# Define the 2 Datasets EMPA and Laser as per the layout in the assignment
# For EMPA - there are missing columns - NiO, F, V2O3, ZnO, Li2O
# But our focus is on the [Ali Spreadsheet]- the main sheet
EMPA_COLS = [
    "CaO", "SiO2", "Cr2O3", "Na2O", "TiO2",
    "MnO", "MgO", "FeO", "Al2O3", "K2O"
]

LASER_COLS = [
    "Mg#", "Li", "Be", "B", "Mg", "Si", "Ca", "Sc", "Ti", "V", "Cr",
    "Mn", "Co", "Ni", "Cu", "Rb", "Sr", "Y", "Zr", "Nb", "Cs", "Ba", "La",
    "Ce", "Pr", "Nd", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb",
    "Lu", "Hf", "Ta", "Pb", "Th", "U"
]

# Performing a Dtype check on them before cleaning
print(df[EMPA_COLS].dtypes)
print(df[LASER_COLS].dtypes)

CaO      float64
SiO2     float64
Cr2O3     object
Na2O     float64
TiO2     float64
MnO      float64
MgO      float64
FeO      float64
Al2O3    float64
K2O      float64
dtype: object
Mg#    float64
Li      object
Be      object
B       object
Mg     float64
Si     float64
Ca     float64
Sc     float64
Ti     float64
V      float64
Cr      object
Mn     float64
Co     float64
Ni      object
Cu      object
Rb      object
Sr     float64
Y      float64
Zr     float64
Nb     float64
Cs      object
Ba      object
La     float64
Ce     float64
Pr     float64
Nd     float64
Sm     float64
Eu     float64
Gd     float64
Tb     float64
Dy     float64
Ho     float64
Er     float64
Tm     float64
Yb     float64
Lu     float64
Hf     float64
Ta     float64
Pb      object
Th     float64
U      float64
dtype: object


In [ ]:
# Examined all columns with dtypes = object and found that there were the following conditions
# 'CPX' value in Cr203 column and a lot of '<' types of data in the Laser columns.
# Decided to have a simple clean_up value function for doing this
# a special handling was needed for the '<' value.
# For this - using the best practice in geochemistry - LOD - [Limit of detection divided by 2]
def clean_value(x):
    if pd.isna(x): #if x is missing, return nan
        return np.nan
    x = str(x).strip() #converts x to string and removes spaces

    # Case 1 : CPX - non numeric - CPX
    if x.upper() == 'CPX': #if uppercase and CPX then print
        print('CPX')
        return np.nan #missing values
    # Case 2 : < below a number value
    # using regex
    # ^ - start of string, < - A literal < char, \s* - 0 or more spaces after <
    # \d+ - one or more digits (integer),
    # (\.\d+)? - optional decimal ; \. literal dot, \d+ - one or more digits. ? - optional
    # $ end of string
    if re.match(r"^<\s*\d+(\.\d+)?$", x):
        num = float(x.replace('<', '').strip()) #remove < and spaces and change to float
        return num/2 # using the LOD/2 method
    # normal numeric
    try:
        return float(x)
    except:
        return np.nan

# applying the above function to both the EMPA and LASER_COLS
for col in EMPA_COLS:
    df[col] = df[col].apply(clean_value)

for col in LASER_COLS:
    df[col] = df[col].apply(clean_value)

# Quick verification
print('Verifying EMPA_COLS - cleanup\n')
print(df[EMPA_COLS].dtypes)
print('Verifying LASER_COLS - cleanup\n')
print(df[LASER_COLS].dtypes)


CPX
Verifying EMPA_COLS - cleanup

CaO      float64
SiO2     float64
Cr2O3    float64
Na2O     float64
TiO2     float64
MnO      float64
MgO      float64
FeO      float64
Al2O3    float64
K2O      float64
dtype: object
Verifying LASER_COLS - cleanup

Mg#    float64
Li     float64
Be     float64
B      float64
Mg     float64
Si     float64
Ca     float64
Sc     float64
Ti     float64
V      float64
Cr     float64
Mn     float64
Co     float64
Ni     float64
Cu     float64
Rb     float64
Sr     float64
Y      float64
Zr     float64
Nb     float64
Cs     float64
Ba     float64
La     float64
Ce     float64
Pr     float64
Nd     float64
Sm     float64
Eu     float64
Gd     float64
Tb     float64
Dy     float64
Ho     float64
Er     float64
Tm     float64
Yb     float64
Lu     float64
Hf     float64
Ta     float64
Pb     float64
Th     float64
U      float64
dtype: object


In [ ]:
# Now that we have cleared the non-numeric issue, we can now check for missingness in the data
empa_missing = pd.DataFrame({
    "Missing_Count": df[EMPA_COLS].isna().sum(),
    "Missing_Percent": df[EMPA_COLS].isna().mean()*100
})

laser_missing = pd.DataFrame({
    "Missing_Count": df[LASER_COLS].isna().sum(),
    "Missing_Percent": df[LASER_COLS].isna().mean()*100
})
print('EMPA Missing Values')
print(empa_missing)
print('LASER Missing Values')
print(laser_missing)

EMPA Missing Values
       Missing_Count  Missing_Percent
CaO                0         0.000000
SiO2               0         0.000000
Cr2O3              1         0.153139
Na2O               0         0.000000
TiO2               0         0.000000
MnO                0         0.000000
MgO                0         0.000000
FeO                0         0.000000
Al2O3              0         0.000000
K2O                0         0.000000
LASER Missing Values
     Missing_Count  Missing_Percent
Mg#              0         0.000000
Li             464        71.056662
Be             464        71.056662
B              464        71.056662
Mg             464        71.056662
Si             464        71.056662
Ca             464        71.056662
Sc             464        71.056662
Ti             464        71.056662
V              464        71.056662
Cr             464        71.056662
Mn             464        71.056662
Co             464        71.056662
Ni             464        71.056662
C

In [ ]:
# This tells us that we have 0.15% missing values in Cr203 column of the EMPA dataset
# And many missing values from the Laser Dataset(71%) with the exception of Mg#
# Saving this 'clean' data for use in the Data Visualization codes
# And for tracability purpose.

df.to_csv(DATA_CLEAN / "df_cleaned.csv", index=False)

In [ ]:
##########################################################################
# Further Data preparation before its use for modeling
# - using the cleaned data to prepare the data for modeling
# - Perform the train-test split
# - Apply SimpleImputer using Median
# - Apply Scaling to normalize the values
# - Prepare the Datasets for Modelling
#######################################################################

# Create the DATA_PREP folder for the output files
# This is for trace-ability

DATA_PREP  = ROOT / "Data_Prepared"
DATA_PREP.mkdir(parents=True, exist_ok=True)

In [ ]:
# -------------------------------
# Create a generic function to prepare data for model training
# - For Classification models and Regression models
# - Imputer and scaling is done only after train-test split
# - to avoid data leakage
# - stratify is True for Classification; False for regression
# -------------------------------
def prepare_features(X, y, stratify=True):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42,
        stratify=y if stratify else None
    ) #stratify=true split data using y to keep class balance; stratify for class balancing

    imputer = SimpleImputer(strategy="median") #get the most occuring value incase of outliers which causes mean value to run
    X_train = imputer.fit_transform(X_train)
    X_test  = imputer.transform(X_test)

    scaler = StandardScaler() #makes big and small values on the same scale, to prevent features with large values from dominating smaller ones
    X_train = scaler.fit_transform(X_train) #Model learns properly (0 to 1)
    X_test  = scaler.transform(X_test) #prevent dataleakage so no fit

    return X_train, X_test, y_train.values, y_test.values

##################################################################
# CREATE 3 Datasets for Classification Models
# 1 - EMPA (653 rows)
# 2 - Laser (189 rows)
# 3 - Combined (EMPA + LASER) - (189 rows)
#################################################################

#
# 1 - Create EMPA - 653 rows
#
# ['T'] column has to be remaped from [1,2,3] to [0,1,2]
# for the Neural Network to work

X_empa = df[EMPA_COLS]
y_empa = df["T"].map({1:0, 2:1, 3:2}) #sparse_categorical_crossentropy expects class labels as zero-based integers: 0 1 2 not 1 2 3

#
# 2 - Create Laser data - 189 rows
# Decided to drop the 71% missing rows of data
# rather than try to create them synthetically
# which could lead to severe bias and mislead the model
#
df_laser = df[LASER_COLS + ["T"]].dropna()
X_laser = df_laser[LASER_COLS]
y_laser = df_laser["T"].map({1:0, 2:1, 3:2})

#
# 3 - Create Combined Dataset comprising EMPA (189 rows) & Laser (189 rows)
#
df_comb = df.loc[df_laser.index, EMPA_COLS + LASER_COLS + ["T"]] #Create a new DataFrame using selected rows and selected columns.
X_comb = df_comb[EMPA_COLS + LASER_COLS]
y_comb = df_comb["T"].map({1:0, 2:1, 3:2})

#
# Generates the 3 sets of Data for training the Classification Models
# all imputed & scaled and ready for use in training the Classification Models
#
# 1st Set - EMPA - 653 rows
X_empa_train, X_empa_test, y_empa_train, y_empa_test = prepare_features(X_empa, y_empa)
# 2nd Set - Laser - 189 rows
X_laser_train, X_laser_test, y_laser_train, y_laser_test = prepare_features(X_laser, y_laser)
# 3rd set - Combined - EMPA & Laser - 189 rows
X_comb_train, X_comb_test, y_comb_train, y_comb_test = prepare_features(X_comb, y_comb)

# Saving the prepared datasets for use in ModelTraining.ipynb script
#
# 1st Set EMPA (653 rows)
np.save(DATA_PREP / "X_empa_train.npy", X_empa_train)
np.save(DATA_PREP / "X_empa_test.npy",  X_empa_test)
np.save(DATA_PREP / "y_empa_train.npy", y_empa_train)
np.save(DATA_PREP / "y_empa_test.npy",  y_empa_test)
# 2nd set Laser (189 rows)
np.save(DATA_PREP / "X_laser_train.npy", X_laser_train)
np.save(DATA_PREP / "X_laser_test.npy",  X_laser_test)
np.save(DATA_PREP / "y_laser_train.npy", y_laser_train)
np.save(DATA_PREP / "y_laser_test.npy",  y_laser_test)
# 3rd set Combined - EMPA & Laser (189 rows)
np.save(DATA_PREP / "X_comb_train.npy", X_comb_train)
np.save(DATA_PREP / "X_comb_test.npy",  X_comb_test)
np.save(DATA_PREP / "y_comb_train.npy", y_comb_train)
np.save(DATA_PREP / "y_comb_test.npy",  y_comb_test)

print("All prepared datasets saved.")

###############################################################
# Extension - made for Linear Regression where Target is Mg#
# Prepared 3 Datasets
# 1 - EMPA (653 rows)
# 2 - Laser (189 rows)
# 3 - Combined (EMPA & Laser rows)
##############################################################

#############################################################
# 1 - Create EMPA (653 rows) Dataset for Regression
#
df_empa_full = df[EMPA_COLS + ["Mg#"]].copy() #.copy makes a safe copy so changes don’t affect the original DataFrame.

X = df_empa_full[EMPA_COLS].copy()
y = df_empa_full["Mg#"].copy()

# Prepare the sets of EMPA data for the Linear Regression Model
# using the same prepare_features function but this round stratify is set to False
# All imputed and scaled and ready for use in training the Regression Model
#
X_empa_reg_train, X_empa_reg_test, y_empa_reg_train, y_empa_reg_test = prepare_features(
    X, y, stratify=False
) #stratify false because continous value doesn't need balancing

# Saving the prepared EMPA datasets for ModelTraining.ipynb script

np.save(DATA_PREP / "X_empa_reg_train.npy", X_empa_reg_train)
np.save(DATA_PREP / "X_empa_reg_test.npy", X_empa_reg_test)
np.save(DATA_PREP / "y_empa_reg_train.npy", y_empa_reg_train)
np.save(DATA_PREP / "y_empa_reg_test.npy", y_empa_reg_test)

print("Saved EMPA FULL regression dataset.")

###################################################################
# 2 - Create Laser Dataset (189 rows)
#
# setting laser_features without column Mg#
laser_features = [c for c in LASER_COLS if c != "Mg#"]

# This is to help with column filtering when we want to get Mg# as the Target
df_laser_reg = df[laser_features + ["Mg#"]].dropna().copy()

X = df_laser_reg[laser_features].copy()
y = df_laser_reg["Mg#"].copy()

# Preparing the sets of laser data for the Linear Regression Model
# Using the same prepare_features function with stratify set to False
# All imputed and scaled for use in training the Linear Regression Model
X_laser_reg_train, X_laser_reg_test, y_laser_reg_train, y_laser_reg_test = prepare_features(
    X, y, stratify=False
)

# Saving the prepared Laser datasets for ModelTraining.ipynb script

np.save(DATA_PREP / "X_laser_reg_train.npy", X_laser_reg_train)
np.save(DATA_PREP / "X_laser_reg_test.npy", X_laser_reg_test)
np.save(DATA_PREP / "y_laser_reg_train.npy", y_laser_reg_train)
np.save(DATA_PREP / "y_laser_reg_test.npy", y_laser_reg_test)
print("Saved LASER regression dataset.")

############################################################################
# 3- Create the Combined EMPA & Laser Dataset (189 rows)
#
# Rows where ALL Laser measurements exist",
laser_complete = df[LASER_COLS].notna().all(axis=1)

# df.loc uses laser_complete to selects only rows where laser data is available
# then selects the required EMPA nad Laser features corresponding to laser_complete rows
df_comb_reg = df.loc[laser_complete, EMPA_COLS + laser_features + ["Mg#"]].dropna().copy() #Create a new DataFrame using selected rows and selected columns.

X = df_comb_reg[EMPA_COLS + laser_features].copy()
y = df_comb_reg["Mg#"].copy()

#
# Prepare the sets of Combined Data for the Linear Regression Model
# Using same prepare_features function but with stratify=False for regression
# All imputed and scaled for use in training the Linear Regression Model
X_comb_reg_train, X_comb_reg_test, y_comb_reg_train, y_comb_reg_test = prepare_features(
    X, y, stratify=False
)

# Saving the prepared Combined datasets for ModelTraining.ipynb script

np.save(DATA_PREP / "X_comb_reg_train.npy", X_comb_reg_train)
np.save(DATA_PREP / "X_comb_reg_test.npy", X_comb_reg_test)
np.save(DATA_PREP / "y_comb_reg_train.npy", y_comb_reg_train)
np.save(DATA_PREP / "y_comb_reg_test.npy", y_comb_reg_test)

print("Saved COMBINED regression dataset.")

##################################################################
#  END OF STAGE 1 - Data Preparation
##################################################################

All prepared datasets saved.
Saved EMPA FULL regression dataset.
Saved LASER regression dataset.
Saved COMBINED regression dataset.
